In [1]:
import os
os.chdir("/root/MC-GRA_test/MC-GRA/")

In [2]:
import argparse
import mindspore
import numpy as np
import random
from mind_dataset import Dataset
from mindspore import context
from mindspore import ops
from mindspore import Tensor
from models.mind_gcn import GCN, embedding_GCN

[WARNING] ME(6286:140199097742528,MainProcess):2024-03-13-18:27:33.117.646 [mindspore/run_check/_check_version.py:102] MindSpore version 2.2.11 and cuda version 11.3.58 does not match, CUDA version [['10.1', '11.1', '11.6']] are supported by MindSpore officially. Please refer to the installation guide for version matching information: https://www.mindspore.cn/install.


In [3]:
def preprocess(adj, features, labels, preprocess_adj=False, preprocess_feature=False, onehot_feature=False, sparse=False, device='cpu'):
    if preprocess_adj:
        adj_norm = normalize_adj(adj)

    if preprocess_feature:
        features = normalize_feature(features)

    labels = Tensor(labels)
    if sparse:
        adj = sparse_mx_to_torch_sparse_tensor(adj)
        features = sparse_mx_to_torch_sparse_tensor(features)
    else:
        if onehot_feature == True:
            features = ops.eye(features.shape[0])
        else:
            features = Tensor(np.array(features.todense()))
        adj = Tensor(adj.todense())

    return adj, features, labels


In [4]:
def dot_product_decode(Z):
    if args.dataset == 'cora' or args.dataset == 'citeseer' or args.dataset == 'AIDS':
        Z = ops.matmul(Z, Z.t())
        adj = ops.relu(Z-ops.eye(Z.shape[0]))
        adj = ops.sigmoid(adj)

    if args.dataset == 'brazil' or args.dataset == 'usair' or args.dataset == 'polblogs':
        Z = mindquantum.utils.normalize(Z, p=2, dim=1)
        Z = ops.matmul(Z, Z.t())
        adj = ops.relu(Z-torch.eye(Z.shape[0]))

    return adj

In [5]:
def transfer_state_dict(pretrained_dict, model_dict):
    state_dict = {}
    for k, v in pretrained_dict:
        if k in model_dict.keys():
            state_dict[k] = v
    return state_dict


In [6]:
parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=15, help='Random seed.')
parser.add_argument('--epochs', type=int, default=100,
                    help='Number of epochs to optimize in GraphMI attack.')
parser.add_argument('--lr', type=float, default=-2.5,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')
parser.add_argument('--nlayers', type=int, default=2,
                    help="number of layers in GCN.")
parser.add_argument('--arch', type=str,
                    choices=["gcn", "gat", "sage"], default='gcn')
parser.add_argument('--dataset', type=str, default='cora',
                    choices=['cora', 'cora_ml', 'citeseer', 'polblogs', 'pubmed', 'AIDS', 'usair', 'brazil', 'chameleon', 'ENZYME', 'squirrel', 'ogb_arxiv'], help='dataset')
# citseer pubmed
parser.add_argument('--density', type=float,
                    default=10000000.0, help='Edge density estimation')
parser.add_argument('--model', type=str, default='PGD',
                    choices=['PGD', 'min-max'], help='model variant')
parser.add_argument('--nlabel', type=float, default=1.0)
parser.add_argument('--iter', type=int, help="iterate times", default=1)
parser.add_argument('--max_eval', type=int,
                    help="max evaluation times for searching", default=100)
parser.add_argument('--log_name', type=str,
                    help="file name to save the result", default="result.txt")
parser.add_argument("--mode", type=str, default="evaluate", choices=["evaluate", "search", "baseline", "ensemble", "aux",
                                                                     "draw_violin", "notrain_test", "dev", "ensemble_search", "gaussian", "gcn_attack"])
parser.add_argument("--measure", type=str, default="MSELoss",
                    choices=["HSIC", "MSELoss", "KL", "KDE", "CKA", "DP"])
parser.add_argument("--measure2", type=str, default="HSIC",
                    choices=["HSIC", "MSELoss", "KL", "KDE", "CKA", "DP"])
parser.add_argument("--nofeature", action='store_true')

parser.add_argument('--weight_aux', type=float, default=0,
                    help="the weight of auxiliary loss")
parser.add_argument('--weight_sup', type=float, default=0,
                    help="the weight of supervised loss")
parser.add_argument('--w1', type=float, default=0.1)
parser.add_argument('--w2', type=float, default=0.1)
parser.add_argument('--w3', type=float, default=0)
parser.add_argument('--w4', type=float, default=0)
parser.add_argument('--w5', type=float, default=0)
parser.add_argument('--w6', type=float, default=6000)
parser.add_argument('--w7', type=float, default=100)
parser.add_argument('--w8', type=float, default=0)
parser.add_argument('--w9', type=float, default=1000)
parser.add_argument('--w10', type=float, default=0)
parser.add_argument('--eps', type=float, default=0,help="eps for adding noise")
parser.add_argument('--useH_A', action='store_true', default=True)
parser.add_argument('--useY_A', action='store_true')
parser.add_argument('--useY', action='store_true')
parser.add_argument('--ensemble', action='store_true')
parser.add_argument('--add_noise', action='store_true')
parser.add_argument('--defense', action='store_true')
args = parser.parse_args(args=[])

In [9]:
from copy import deepcopy

context.set_context(mode=context.GRAPH_MODE, device_target="CPU")

np.random.seed(args.seed)
random.seed(args.seed)
data = Dataset(root='./dataset', name=args.dataset, setting='GCN')
adj, features, labels, init_adj = data.adj, data.features, data.labels, data.init_adj



idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test

# choose the target nodes
idx_attack = np.array(random.sample(
    range(adj.shape[0]), int(adj.shape[0]*args.nlabel)))

num_edges = int(0.5 * args.density * adj.sum() /
                adj.shape[0]**2 * len(idx_attack)**2)
adj, features, labels = preprocess(adj, features, labels, preprocess_adj=False, onehot_feature=False)
feature_adj = dot_product_decode(features)
if args.nofeature:
    feature_adj = ops.eye(*feature_adj.size())
init_adj = Tensor(init_adj.todense())
if args.arch == "gcn":
    victim_model = GCN(nfeat=features.shape[1], nclass=labels.max().item() + 1, nhid=16, idx_train=idx_train, nlayer=args.nlayers,
                       dropout=0.5, weight_decay=5e-4)
    if args.defense:
        victim_model.load_state_dict(torch.load(f'./defense/{args.dataset}_{args.arch}_{args.nlayer}.pt', map_location=device))
        victim_model = victim_model
    else:
        victim_model = victim_model
        victim_model.fit(features, adj, labels, idx_val)

    embedding = embedding_GCN(
        nfeat=features.shape[1], nhid=16, nlayer=args.nlayers)
    
    mindspore.load_param_into_net(embedding, transfer_state_dict(
        victim_model.parameters_dict().items(), embedding.parameters_dict()), strict_load=False)
    
   # embedding.load_state_dict(transfer_state_dict(
     #   victim_model.state_dict(), embedding.state_dict()))

    embedding.gc = deepcopy(victim_model.gc)

Loading cora dataset...
=== training gcn model ===
Epoch 0, training loss: 1.92416512966156 , val acc: 0.3084112
Epoch 10, training loss: 1.2542532682418823 , val acc: 0.6853582
=== picking the best model according to the performance on validation ===


[WARNING] ME(6286:140199097742528,MainProcess):2024-03-13-18:33:41.382.457 [mindspore/train/serialization.py:1378] For 'load_param_into_net', 2 parameters in the 'net' are not loaded, because they are not in the 'parameter_dict', please check whether the network structure is consistent when training and loading checkpoint.
[WARNING] ME(6286:140199097742528,MainProcess):2024-03-13-18:33:41.383.897 [mindspore/train/serialization.py:1383] gc1.weight is not loaded.
[WARNING] ME(6286:140199097742528,MainProcess):2024-03-13-18:33:41.384.912 [mindspore/train/serialization.py:1383] gc1.bias is not loaded.


In [ ]:
%whos

In [ ]:
preds = features.max(1)

In [ ]:
features[Tensor(idx_train)]

In [ ]:
print(victim_model.parameters_dict())

In [ ]:
params=victim_model.parameters_dict().items()
for k, v in params:
    print(k)